In [1]:
%matplotlib qt
import time
import sys
import math
import os
import numpy as np
import pandas as pd
import MySQLdb
from datetime import datetime
import calendar
import holidays
import matplotlib.pyplot as plt
sys.path.insert(0, '/Users/kenleejr92/energy_market_project/scripts/MySQL_scripts')
from Query_ERCOT_DB import Query_ERCOT_DB
import cPickle as pickle
from datetime import datetime
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.mllib.linalg import Matrix, Matrices
import pyspark.mllib.linalg.distributed as pydist
from pyspark.sql import SQLContext
from pyspark.mllib.stat import Statistics
from sklearn.cluster import AffinityPropagation
from sklearn import preprocessing
import mpld3
import re
from sets import Set
mpld3.enable_notebook()

class LMP_Query(Query_ERCOT_DB):

    def __init__(self):
        self.SPPs = ['HB_BUSAVG', 
                     'HB_HOUSTON', 
                     'HB_HUBAVG', 
                     'HB_NORTH', 
                     'HB_SOUTH', 
                     'HB_WEST', 
                     'LZ_AEN', 
                     'LZ_CPS', 
                     'LZ_HOUSTON', 
                     'LZ_LCRA', 
                     'LZ_NORTH', 
                     'LZ_RAYBN', 
                     'LZ_SOUTH',  
                     'LZ_WEST']   
        self.node_dict = {}
        self.table_list = []
        self.df = None
        self.CRR_nodes = None
        self.table_boundaries = {'table0':('0001', 'BLUEMD1_8X'),
                                 'table1':('BLUEMD1_8Z', 'CHT_M'),
                                 'table2':('CHT_X', 'DUKE_8405'),
                                 'table3':('DUKE_8505', 'ELEVEE_E8'),
                                 'table4':('ELEVEE_W8', 'GREENLK_L_A'),
                                 'table5':('GREENLK_L_B', 'KEETER'),
                                 'table6':('KEITH', 'L_CEDAHI8_1Y'),
                                 'table7':('L_CEDAHI8_1Z', 'MOSES_1G'),
                                 'table8':('MOSES_2G', 'PHR_8135'),
                                 'table9':('PHR_8140', 'SANDOW1_8Y'),
                                 'table10':('SANDOW_4G', 'TCN7225_BUS'),
                                 'table11': ('TCN7230_BUS', 'VENSW_1777'),
                                 'table12':('VENSW_1785', '_WC_V_C')
                                 }
        for i in range(0,13):
            Query_ERCOT_DB.c.execute("""SHOW columns FROM DAM_LMP%s""" % i)
            result = [r[0] for r in Query_ERCOT_DB.c.fetchall()[2:]]
            self.table_list.append(result)
            for node in result:
                self.node_dict[node] = i
        
    
    def get_CRR_nodes(self):
        Query_ERCOT_DB.c.execute("""SELECT DISTINCT Sink FROM crr_ownership ORDER BY Sink""")
        nodes = list(Query_ERCOT_DB.c.fetchall())
        CRR_prefixes = [r[0] for r in nodes]
        pattern = re.compile('.*_')
        matching_patterns = Set()
        for idx, node in enumerate(CRR_prefixes):
            matches = re.findall(pattern, node)
            if matches: matching_patterns.add(matches[0][:-1])
        # flatten list of lists      
        all_nodes = [item for sublist in self.table_list for item in sublist] + self.SPPs
        self.CRR_nodes = []
        for pattern in matching_patterns:
            pattern2 = re.compile('(.*%s.*)' % pattern)
            for node in all_nodes:
                if re.search(pattern2, node):
                    self.CRR_nodes.append(node)
        return self.CRR_nodes
    
    def query_single_node(self, node):
        s="""SELECT delivery_date, hour_ending, %s from DAM_LMP%s order by delivery_date, hour_ending""" % (node, self.node_dict[node])
        Query_ERCOT_DB.c.execute(s)
        result = list(Query_ERCOT_DB.c.fetchall())
        fresult = []
        for r in result:
            temp = ()
            date = r[0]
            time = str(int(r[1].split(":")[0])-1)
            dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
            for x in r[2:]:
                if x == None: x = 0
                temp = temp + (float(x),)
            r = (dt,) + temp
            fresult.append(r)
        self.df = pd.DataFrame(data=[f[1:] for f in fresult], index=[r[0] for r in fresult], columns=[node])
        return self.df
        
    def query(self, nodes, start_date='2011-01-01', end_date='2015-5-23'):
        node_string=''
        for node in nodes:
            node_string = node_string + node + ',' + ' '
        node_string = node_string[:-2]
        s = """SELECT DAM_LMP0.delivery_date, DAM_LMP0.hour_ending, %s 
                                    from DAM_LMP0 join DAM_LMP1 on (DAM_LMP0.delivery_date = DAM_LMP1.delivery_date and DAM_LMP0.hour_ending = DAM_LMP1.hour_ending) 
                                    join DAM_LMP2 on (DAM_LMP0.delivery_date = DAM_LMP2.delivery_date and DAM_LMP0.hour_ending = DAM_LMP2.hour_ending)
                                    join DAM_LMP3 on (DAM_LMP0.delivery_date = DAM_LMP3.delivery_date and DAM_LMP0.hour_ending = DAM_LMP3.hour_ending)
                                    join DAM_LMP4 on (DAM_LMP0.delivery_date = DAM_LMP4.delivery_date and DAM_LMP0.hour_ending = DAM_LMP4.hour_ending)
                                    join DAM_LMP5 on (DAM_LMP0.delivery_date = DAM_LMP5.delivery_date and DAM_LMP0.hour_ending = DAM_LMP5.hour_ending)
                                    join DAM_LMP6 on (DAM_LMP0.delivery_date = DAM_LMP6.delivery_date and DAM_LMP0.hour_ending = DAM_LMP6.hour_ending)
                                    join DAM_LMP7 on (DAM_LMP0.delivery_date = DAM_LMP7.delivery_date and DAM_LMP0.hour_ending = DAM_LMP7.hour_ending)
                                    join DAM_LMP8 on (DAM_LMP0.delivery_date = DAM_LMP8.delivery_date and DAM_LMP0.hour_ending = DAM_LMP8.hour_ending)
                                    join DAM_LMP9 on (DAM_LMP0.delivery_date = DAM_LMP9.delivery_date and DAM_LMP0.hour_ending = DAM_LMP9.hour_ending)
                                    join DAM_LMP10 on (DAM_LMP0.delivery_date = DAM_LMP10.delivery_date and DAM_LMP0.hour_ending = DAM_LMP10.hour_ending)
                                    join DAM_LMP11 on (DAM_LMP0.delivery_date = DAM_LMP11.delivery_date and DAM_LMP0.hour_ending = DAM_LMP11.hour_ending)
                                    join DAM_LMP12 on (DAM_LMP0.delivery_date = DAM_LMP12.delivery_date and DAM_LMP0.hour_ending = DAM_LMP12.hour_ending)
                                    join DAM_SPPs on (DAM_LMP0.delivery_date = DAM_SPPs.delivery_date and DAM_LMP0.hour_ending = DAM_SPPs.hour_ending)
                                    where DAM_LMP0.delivery_date > "%s" and DAM_LMP0.delivery_date < "%s" order by DAM_LMP0.delivery_date, DAM_LMP0.hour_ending;""" % (node_string, start_date, end_date)
        Query_ERCOT_DB.c.execute(s)
        result = list(Query_ERCOT_DB.c.fetchall())
        fresult = []
        for r in result:
            temp = ()
            date = r[0]
            time = str(int(r[1].split(":")[0])-1)
            dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
            for x in r[2:]:
                if x == None: x = 0
                temp = temp + (float(x),)
            r = (dt,) + temp
            fresult.append(r)
        self.df = pd.DataFrame(data=[f[1:] for f in fresult], index=[r[0] for r in fresult], columns=nodes)
        return self.df
        
    def get_price(self, node, date, hour_ending):
        for i in range(0,13):
            node = append_n(node)
            if node in self.table_columns['table%s' % i]:
                Query_ERCOT_DB.c.execute("""SELECT %s FROM DAM_LMP%s WHERE delivery_date = "%s" AND hour_ending = \"%s\"""" % (node, i, date, hour_ending))
                result = list(Query_ERCOT_DB.c.fetchall())[0][0]
                return result
        

def append_n(name):
    if name[0] in ['0','1','2','3','4','5','6','7','8','9'] or name == 'LOAD':
        name = 'n' + name
    return name

def dist(x,y):
    cost = 0
    for i in range(len(x)):
        if x[i] == 0 or y[i] == 0:
            continue
        else:
            cost = cost + np.abs(y[i] - x[i])[0]
    return cost

def string_to_date(string_date):
    return datetime.strptime(string_date, "%Y-%m-%d %H")

def date_to_string(date):
    return date.strftime("%Y-%m-%d %H")

def weekday_of_date(date):
    return calendar.day_name[date.weekday()]

def work_day_or_holiday(date):
    us_holidays = holidays.UnitedStates()
    if date in us_holidays or weekday_of_date(date) == "Sunday" or weekday_of_date(date) == "Saturday":
        return int(1)
    else: return int(0)

/Users/kenleejr92/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: DeprecationWarning: the sets module is deprecated


In [2]:
conf = SparkConf().setMaster("local").setAppName("spark_DAM_correlation")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [2]:
qcrr = LMP_Query()
if not os.path.isfile('crr_nodes.pkl'):
    print('making new')
    crr_nodes = qcrr.get_CRR_nodes()
    f = open('crr_nodes.pkl', 'w+')
    pickle.dump(crr_nodes, f)
    f.close()
else:
    print('retrieving saved')
    f = open('crr_nodes.pkl', 'r')
    crr_nodes = pickle.load(f)
    f.close()

retrieving saved


In [ ]:
if not os.path.isfile('crr_node_prices.pkl'):
    print('making new')
    x1 = qcrr.query(crr_nodes[0:500]).as_matrix()
    x2 = qcrr.query(crr_nodes[500:1000]).as_matrix()
    x3 = qcrr.query(crr_nodes[1000:1500]).as_matrix()
    x4 = qcrr.query(crr_nodes[1500:2000]).as_matrix()
    x5 = qcrr.query(crr_nodes[2000:2500]).as_matrix()
    x6 = qcrr.query(crr_nodes[2500:3000]).as_matrix()
    x7 = qcrr.query(crr_nodes[3000:]).as_matrix()
    x8 = np.concatenate((x1,x2), axis=1)
    x8 = np.concatenate((x8,x3), axis=1)
    x8 = np.concatenate((x8,x4), axis=1)
    x8 = np.concatenate((x8,x5), axis=1)
    x8 = np.concatenate((x8,x6), axis=1)
    x8 = np.concatenate((x8,x7), axis=1)
    f = open('crr_node_prices.pkl', 'w+')
    pickle.dump(x8, f)
    crr_prices = x8
    f.close()
else:
    print('retrieving saved')
    f = open('crr_node_prices.pkl', 'r')
    crr_prices = pickle.load(f)
    f.close()

In [3]:
if not os.path.isfile('sim_mat.pkl'):
    sample_size = crr_prices.shape[0]
    train_size = 0.8
    train_split = int(0.8*sample_size)
    train_set = crr_prices[0:train_split,:]
    sim_mat = np.zeros((len(crr_nodes),len(crr_nodes)))
    for i in range(len(crr_nodes)):
        for j in np.arange(i,len(crr_nodes)):
            sim_mat[i,j] = np.negative(np.sum(np.square(train_set[:,i] - train_set[:,j])))

    for i in range(len(crr_nodes)):
        for j in np.arange(i,len(crr_nodes)):
            sim_mat[j,i] = sim_mat[i,j]
    with open('sim_mat.pkl', 'w+') as f: pickle.dump(sim_mat, f)
else: 
    with open('sim_mat.pkl', 'r') as f: sim_mat = pickle.load(f)

af = AffinityPropagation(affinity='precomputed').fit(sim_mat)

KeyboardInterrupt: 

In [4]:
x_df = pd.DataFrame.from_csv('/Users/kenleejr92/energy_market_project/crr_node_correlation_matrix.csv', header=None, index_col=None)
x = x_df.as_matrix()
for i, item in enumerate(x[:,0]):
    x[i,0] = float(item[1:])
for i, item in enumerate(x[:,3587]):
    x[i,3587] = float(item[:-1])
    

In [3]:
col_dict = {}
for i, col in enumerate(crr_nodes):
    col_dict[i] = col
if not os.path.isfile('cluster_list.pkl'):
    cluster_list = [[] for i in range(af.cluster_centers_indices_.size)]
    for idx, item in enumerate(af.labels_):
        cluster_list[item].append(col_dict[idx])
    f = open('cluster_list.pkl', 'w+')
    pickle.dump(cluster_list, f)
    f.close()
else:
    f = open('cluster_list.pkl', 'r')
    cluster_list = pickle.load(f)
    f.close()
    
if not os.path.isfile('exemplars.pkl'):
    exemplars = []
    for idx, cluster in enumerate(cluster_list):
        exemplars.append(col_dict[idx])
    f = open('exemplars.pkl', 'w+')
    pickle.dump(exemplars, f)
    f.close()
else:
    f = open('exemplars.pkl', 'r')
    exemplars = pickle.load(f)
    f.close()

df1 = qcrr.query(exemplars)
mat = df1.as_matrix()
non_zero_exemplars = []
for i in range(mat.shape[1]):
    series = mat[:,i]
    zero_count=0
    for j in series:
        if j==0.0:
            zero_count+=1
    if zero_count<500:
        non_zero_exemplars.append(exemplars[i])
df = df1[non_zero_exemplars]
# ax = df.plot()
# ax.set_xlabel('Hour')
# ax.set_ylabel('Dollar/MWh')
# ax.legend_.remove()
# plt.show()

In [20]:
exemplars = non_zero_exemplars
mat = df[exemplars].as_matrix()
train_size = 0.8
sample_size = mat.shape[0]
train_split = int(0.8*sample_size)
exemplar_prices = mat[0:train_split,:]
scaler = preprocessing.MinMaxScaler((0,1))
exemplar_prices = scaler.fit_transform(exemplar_prices)
medians = np.median(exemplar_prices, axis=0)
def quantize(a):
    a_med = np.median(a)
    for idx, i in enumerate(a):
        if i >= a_med:
            a[idx] = 1.0
        else: a[idx] = 0.0
    return a
exemplar_prices = np.apply_along_axis(quantize, 0, exemplar_prices)

def MI(a,b):
    p00 = 0.0
    p10 = 0.0
    p01 = 0.0
    p11 = 0.0
    MI = 0.0
    zero=np.float64(0.0)
    one=np.float64(1.0)
    for i in range(len(a)):
        if a[i]+b[i]==0.0:
            p00 += 1.0
        if a[i]==zero and b[i]==one:
            p10 += 1.0
        if (a[i]==one) and (b[i]==zero):
            p01 += 1.0
        if a[i]+b[i]==2.0:
            p11 += 1.0
    p00 = p00/train_split
    p10 = p10/train_split
    p01 = p01/train_split
    p11 = p11/train_split
    if p00==0 or p10==0 or p01==0 or p11 ==0:
        return 0.0
    else: return p00*math.log(p00/0.25,2)+\
           p10*math.log(p10/0.25,2)+\
            p01*math.log(p01/0.25,2)+ p11*math.log(p11/0.25,2)

e = df.columns[0]
print('feature selection for %s' % e)
past_index = [24,25,47,48]
features = []
targets = []
feature_labels=[]
for c in df.columns:
    for j in past_index:
        feature_labels.append(c + '_' + str(j))
for pred_hour_index in range(exemplar_prices.shape[0]):
    temp_features = []
    if type(pred_hour_index) == slice: continue
    if pred_hour_index - 48 >= 0:
        targets.append(exemplar_prices[pred_hour_index, 0])
        for i,c in enumerate(df.columns):
            temp_features += [exemplar_prices[pred_hour_index-j, i] for j in past_index]
        features.append(temp_features)

targets = np.array(targets)
features = np.array(features)

MIs = []
for i in range(features.shape[1]):
    MIs.append((feature_labels[i],MI(targets, features[:,i])))

feature selection for CAPRIDGE_CR1


In [22]:
MIs.sort(key=lambda tup: tup[1], reverse=True) 
selected = filter(lambda x: x[1]>0.4, MIs)

f = open('%s_MIs.pkl' % e,'w+')
pickle.dump(selected, f)
f.close()


In [80]:
f=open('CAPRIDGE_CR1_MIs.pkl','r')
MIs = pickle.load(f)
f.close()
selected = filter(lambda x: x[1]>0.4, MIs)

In [ ]:
df=qcrr.query(exemplars,end_date='2016-5-23')

In [82]:
targets = []
features = []
e = exemplars[0]
nodes=[]
for s in selected:
    nodes.append(s[0][:-3])
nodes = list(set(nodes))
mat=df.as_matrix()
for pred_hour_index in range(mat.shape[0]):
    temp_features = []
    if type(pred_hour_index) == slice: continue
    if pred_hour_index - 24 >= 0:
        targets.append(mat[pred_hour_index, 0])
        for i,c in enumerate(nodes):
            temp_features += [mat[pred_hour_index-24, i]]
        features.append(temp_features)

targets = np.array(targets)
features = np.array(features)

f=open('features.pkl', 'w+')
pickle.dump(features, f)
f.close()
f=open('targets.pkl', 'w+')
pickle.dump(targets, f)
f.close()

In [74]:
features = np.array(features)
targets.shape

(38419,)